In [ ]:
%%R


In [ ]:
from preamble import *
plt.close('all')
codpy_param = {'rescale:xmax': 1000,
'rescale:seed':42,
'sharp_discrepancy:xmax':1000,
'sharp_discrepancy:seed':30,
'sharp_discrepancy:itermax':5,
'discrepancy:xmax':500,
'discrepancy:ymax':500,
'discrepancy:zmax':500,
'discrepancy:nmax':2000}

# Kernel methods for machine learning

## Aim of this chapter

We will present first our techniques for dealing with problems of learning machine and we cover here two main ingredients that are central in the design of our algorithms. 

* First, all methods described in this chapter depend on a choice of a kernel and use transformation maps acting on basic kernels in order to adapt them to any particular problem.

* Second, this chapter also provides discrete differentiation operators that are relevant for machine learning as well problems involving partial differential operators. 

Importantly, the presented below provides us with the key building blocks of our algorithms and will allow us to formulate more advanced numerical algorithms in the next chapters of this monograph. 

For a precise description of our framework we need some further notation. A set of $N_x$ observable data in $D$ dimensions is available, denoted by the symbol $X \in \mathbb{R}^{N_x\times D}$, together with a $D_f$-dimensional vector-valued function $f(X):\mathbb{R}^{N_x \times D_f}$ are the training values associated with the training variables. The input dataset is therefore 
$$
(X,f(X)) := \{x^n, f(x^n)\}_{n = 1,\dots,N_x}, \quad X \in \mathbb{R}^{N_x \times D}, \qquad f(X) \in \mathbb{R}^{N_x \times D_f}.
$$
We are interested in predicting test values $f(Z):\mathbb{R}^{N_z \times D_f}$ on a new set of variables called *a test set* $Z \in \mathbb{R}^{N_z \times D}$: 
\begin{equation} 
(Z,f_z) := \{z^n, f_z^n\}_{n = 1,\dots,N_z}, \quad Z \in \mathbb{R}^{N_z \times D}, \qquad f_z \in \mathbb{R}^{N_z \times D_f}. (\#eq:PP)
\end{equation}

In all examples and numerical experiments in this section we take a matrix $X \in \mathbb{R}^{N_x \times D}$ and use the following periodic and increasing function:
\begin{equation}
f(X) = \prod_{d=1,\ldots,D} \cos (4\pi x_d) + \sum_{d=1,\ldots,D} x_d, \qquad x \in \mathbb{R}^D
\end{equation}

In [ ]:
def my_fun(x):
    import numpy as np
    from math import pi
    D = len(x)
    res = 1.;
    for d in range(0,D):
        res *= np.cos(2 * x[d] * pi) 
    for d in range(0,D):
        res += x[d]
    return res
    
def nabla_my_fun(x):
    import numpy as np
    from math import pi
    D = len(x)
    a = np.zeros((D))
    cost = 1.;
    for d in range(0,D):
        cost *= np.cos(2 * x[d] * pi) 
    for d in range(0,D):
      a[d] = 1
      if cost != 0.:
        a[d] += 2.* cost * pi*np.sin(2* x[d] * pi) / np.cos(2 * x[d] * pi)
    return a

In [ ]:
set_kernel = kernel_setters.kernel_helper(
kernel_setters.set_tensornorm_kernel, 0,0 ,map_setters.set_unitcube_map)
set_kernel()

We take $X \in \mathbb{R}^{N_x\times D},Z \in \mathbb{R}^{N_z\times D}$, $f(X) \in \mathbb{R}^{N_x\times D_f}$, $f(Z) \in \mathbb{R}^{N_z\times D_f}$. 

<!-- ?????????????????????????????????????????We generate our test data randomly, as will be explained in Section \@ref(general-specification-of-tests). This data generator applies to compute regular, and Cartesian meshes for a better display of the results. The user can run all of our tests removing the "field" type and analyze the effect of choosing randomly generated data; this applies to all the results in the present section. -->
<!-- The basic settings for our test are  -->

In [ ]:
D,Nx,Ny,Nz=2,500,500,500
data_random_generator_ = data_random_generator(fun = my_fun,nabla_fun = nabla_my_fun, types=["cart","cart","cart"])
x,y,z,fx,fy,fz,nabla_fx,nabla_fz,Nx,Ny,Nz =  data_random_generator_.get_raw_data(D=D,Nx=Nx,Ny=Ny,Nz=Nz)

In [ ]:
%%R
knitr::kable(cbind(py$D,py$Nx,py$Ny,py$Nz), col.names = c('$D$', '$N_x$', '$N_y$', '$N_z$'), caption = "Data's dimensions", escape = FALSE) %>%
  kable_styling(latex_options = "HOLD_position")

The case $N_x = N_z$ corresponds to data **extrapolation**, as explained in \@ref(extrapolation-interpolation-and-projection) below. For illustration purposes, we set another set of parameters, corresponding to data **projection**, as explained also in the section \@ref(extrapolation-interpolation-and-projection), i.e. when $N_y<<N_x$. 

In [ ]:
dummy,y1,dummy,dummy,fy1,dummy,dummy,dummy,dummy,dummy,dummy =  data_random_generator_.get_raw_data(D=D,Nx=Nx,Ny=Ny/100,Nz=Nz)

In [ ]:
%%R
knitr::kable(cbind(py$D,py$Nx,length(py$y1),py$Nz), col.names = c('$D$', '$N_x$', '$N_y$', '$N_z$'), caption = "Data's dimensions", escape = FALSE) %>%
  kable_styling(latex_options = "HOLD_position")

In [ ]:
dummy,y2,dummy,dummy,fy2,dummy,dummy,dummy,dummy,dummy,dummy =  data_random_generator_.get_raw_data(D=D,Nx=Nx,Ny=Ny/100,Nz=Nz)

The figure \@ref(fig:xfxyfyzfz) provides us with an example of machine learning settings. We will rely on the first one throughout the following discussion. Here, the left-hand figures is the training set  $(X, f(X))$ of variables and values 
and the righ-hand figure displays the test set $(Z, f(Z))$ of variable and values, while the middle figure shows the parameter set $(Y ,f(Y))$ of variables and values. As mentioned earlier, the latter is a choice, made by a reader, determining not only the overall accuracy, but also computational cost. 

In [ ]:
multi_plot([(x,fx),(y,fy),(z,fz),(x,fx),(y2,fy2),(z,fz),(x,fx),(z,fz),(z,fz)],plot_trisurf,mp_ncols = 3, projection='3d',mp_max_items = -1, f_names=["training set","parameter set (extrapolation)","test set","training set","parameter set (projection)","test set","training set","parameter set (interpolation)","test set"])

## Fundamental notions for supervised learning 

### Preliminaries 

**Positive definite kernels and kernel matrices**. We briefly give a definition of a kernel.

Let $k: \mathbb{R}^D \times \mathbb{R}^D \mapsto \mathbb{R}$ be a symmetric real-valued function, i.e. satisfying $k(x, y)=k(y, x)$. For any two sequences of points $X = x^1, \cdots, x^{N_x} \in \mathbb{R}^D$, $Y = y^1, \cdots, y^{N_y} \in \mathbb{R}^D$ we define a kernel matrix $K(X,Y) := \big(k(x^n,y^m) \big)_{n,m} \in \mathbb{R}^{N_x \times N_y}$
\begin{equation}
    K(X, Y) = 
    \left( 
    \begin{array}{ccc} 
        k(x^1,y^1) & \cdots & k(x^1,y^{N_y}) \\
        \ddots & \ddots & \ddots \\
         k(x^{N_x},y^1) & \cdots & k(x^{N_x},y^{N_y})
    \end{array}
    \right)
\end{equation}
We say that $k$ is a positive-definite kernel if for any sequence of distinct points $X \in \mathbb{R}^{N_x \times D}$ and $c^1,...,c^{N_x} \in \mathbb{R}^{N_x}$:
\begin{equation}
\sum_{i,j \leq  N_x} c^i c^j k(x^i,x^j) \geq 0. 
\end{equation}

If $N_x=N_y$, the matrix $K(X,Y)$ is called a Gram matrix. The dimension of a kernel matrix $K(X,Y)$ is usually $N_x \times N_y$, except for some combined kernels; see the section on kernel engineering \@ref(kernel-engineering).

If $K(X,Y)$ is positive-definite on a certain submanifold, we say that the kernel is **conditionally positive-definite** -- in the sense that it is positive-definite, conditionally to the fact that $X, Y$ belongs to this submanifold.

As should be expected we can not use an arbitrary symmetric function and in our framework, we always use the kernels that are in the class of positive-definite kernels. The available kernels in our library are listed in the table \@ref(tab:249):

In [ ]:
kernels = pd.DataFrame(factories.get_kernel_factory_keys()).reset_index(drop=True) #, columns=[("Kernels")])
kernels1 = kernels[0:4]
kernels2 = kernels[5:9]
kernels3 = kernels[10:14]
kernels4 = kernels[14:18]

In [ ]:
%%R
kernels1 = py$kernels1
kernels2 = py$kernels2
kernels3 = py$kernels3
kernels4 = py$kernels4
rownames(kernels1) <- c()
rownames(kernels2) <- c()
rownames(kernels3) <- c()
rownames(kernels4) <- c()
knitr::kable(cbind(kernels1,kernels2,kernels3,kernels4), caption = "A list of available kernels", col.names = NULL) %>%
  kable_styling(latex_options = "HOLD_position")

Observe that a certain scaling of the kernel may be required in order to handle some input data, which is exactly the purpose of the maps, discussed below.

\begin{example}
Gaussian kernel is used by default in CoDpy library:
\begin{equation} 
k(x, y) = \exp(-\pi|x - y|^2).
(\#eq:Gaussian)
\end{equation}
\end{example}

\begin{example}
Consider the following family of symmetric functions $k(x,y)$ with $x,y \in \mathbb{R}^D$:
$$
k(x,y) = g(<S(x),S(y)>_{\mathbb{R}^P}), \quad S:\mathbb{R}^D \mapsto \mathbb{R}^P, 
$$
where $g$ is called an activation function and $S$ is a mapping. In particular, $k(x,y)=<(1,x,x^Tx,\ldots),(1,y,y^Ty,\ldots)>$ defines a kernel corresponding to a linear regression over a polynomial basis, hence is positive definite, but is not strictly positive. The RELU kernel, given as $k(x, y)=\max(<x,y>+c,0)$ (with $c$ being a constant) is a conditionally positive definite.
\end{example}


In [ ]:
Knm = op.Knm(x,y, set_codpy_kernel = set_kernel, rescale = True)
#print("Knm shape:",Knm.shape)

Let us choose the kernel to be *tensornorm* (again discussed below) and we refer to the section \@ref(brief-overview-of-methods-of-machine-learning) for the description of external parameters in the described kernel method. Finally, we output some values of the kernel matrix induced by Gaussian kernel in the table \@ref(tab:450) computed using Codpy's *op.Knm* function.

In [ ]:
%%R
knitr::kable(py$Knm[0:4,0:4], caption = "First four rows and columns of a kernel matrix $K(X,Y)$")%>%
kable_styling(latex_options = "HOLD_position")

**Inverse kernel matrix**. The inverse of a Kernel matrix is denoted $K(X, Y)^{-1}$, and this inverse is computed, if $X=Y$, as follows:  
$$
  K(X, X)^{-1} = (K(X, X) + \epsilon I_d)^{-1}.
$$
In the general range $X \neq Y$, it is computed using a least-square inversion, namely 
$$
  K(X,Y)^{-1} = (K(Y,X)K(X,Y)+ \epsilon I_d)^{-1}K(Y,X)
$$
We refer to Tikhonov regularization parameter $\epsilon$ as a \textsc{regularization} parameter, and by default takes the value $\epsilon = 10^{-8}$. 

In [ ]:
Knm_inv =op.Knm_inv(x=x,y=y)
inv_shape = Knm_inv.shape

The table \@ref(tab:455) illustrates the first four rows and columns of the kernel matrix's inverse $K(X,Y)^{-1} \in \mathbb{R}^{N_y\times N_x}$, $N_x = N_y$. 

In [ ]:
%%R
knitr::kable(py$Knm_inv[0:4,0:4], caption = "First four rows and columns of an inverted kernel matrix $K(X,Y)^{-1}$")%>%
kable_styling(latex_options = "HOLD_position")

The matrix product $K(X, Y)K(X, Y)^{-1}$ in the table \@ref(tab:455) is just a projection operator. It might not be the identity depending on the experimental setting, for one of the following reasons: 

- If $N_x \neq N_y$.

- If the Tikhonov regularization parameter $\epsilon >0$. One can put $\epsilon = 0$, this choice can be set by the user, but take care of performance issues. If the kernel is not strictly positive, then the library might raise an exception, and switch from a standard inversion of matrix to a non-strictly positive-definite inversion, that can be more computationally costly.

- If the kernel under consideration is such that $K(X, X)K(X, X)^{-1}$ does not have a full rank, for instance if a linear regression kernel is used; see the section on kernel engineering (section \@ref(kernel-engineering)). In which case this matrix is a projection over $Ker(K(X, X))$.

<!-- ```{python} -->
<!-- Id = (np.dot(Knm_inv,Knm))[0:4,0:4] -->
<!-- ``` -->

<!-- ```{r, label= 955, caption = "Matrix Knm: a sampling."} -->
<!-- knitr::kable(py$Id, caption = "Projection operator $P(x,y,x)$") %>% -->
<!--   kable_styling(latex_options = "HOLD_position") -->
<!-- ``` -->

**Distance matrices**. Distance matrix is simple and very handy tool for kernel methods.

Let $k(\cdot,\cdot) : \mathbb{R}^D \times \mathbb{R}^D \mapsto \mathbb{R}$ be a positive-definite kernel. Then the distance function $d_k(x,y)$ for $x \in \mathbb{R}^D, y \in \mathbb{R}^D$ is defined as follows:
\begin{equation}
d_k(x,y) = k(x,x) + k(y,y) - 2k(x,y). (\#eq:ES)
\end{equation}

Note that for a positive-definite kernel the latter expression is continuous, positive, and satisfies $d(x,x) = 0$.

For any two sequences of points $X = x^1,...,x^{N_x} \in \mathbb{R}^D$, $Y = y^1,...,y^{N_y} \in \mathbb{R}^D$ we define a distance matrix $D(X,Y) \in \mathbb{R}^{N_x \times N_y}:$
\begin{equation}
    D(X,Y) = 
    \left( 
    \begin{array}{ccc} 
        d_k(x^1,y^1) & \cdots & d_k(x^1,y^M) \\
        \ddots & \ddots & \ddots \\
         d_k(x^N,y^1) & \cdots & d_k(x^N,y^M)
    \end{array}
    \right)
\end{equation}


In [ ]:
Dnmxy =op.Dnm(x=x,y=y)

The table \@ref(tab:485) outputs the first four columns of the kernel-based distance distance matrix $D(X,Y)$.

In [ ]:
%%R
knitr::kable(py$Dnmxy[0:4,0:4], caption = "First four rows and columns of a kernel-based distance matrix $D(X,Y)$") %>%
kable_styling(latex_options = "HOLD_position")

**CodPy's algorithms**. CodPy's algorithms offer general functions in order to get predictions in \@ref(eq:PP) from the choice of a kernel. More precisely, the following operator (with $A^{-1} := (A^TA)^{-1}A^T$ denoting the least-square inverse)
\begin{equation}
 f_z := \mathcal{P}_{k}(X,Y,Z)f(X):= K(Z,Y)K(X,Y)^{-1}f(X),\quad K(Z,Y) \in \mathbb{R}^{N_z \times N_y}, K(X,Y) \in \mathbb{R}^{N_x \times N_y} (\#eq:P)
\end{equation}
defines a supervised learning machine which we call a **feed-forward** machine. We also consider $\mathcal{P}_{k}(X,Y,Z) \in \mathbb{R}^{N_z \times N_x}$ as a **projection operator** and it is well-defined once a kernel $k$ has been chosen. Observe that two factors arise in \@ref(eq:P), namely the so-called Kernel matrix $K(X,Y)$ (discussed below) and the **projection set of variables** denoted by $Y \in \mathbb{R}^{N_y \times D}$.  To motivate the role of the later, let us consider two particular operators that do not depend upon $Y$: 
\begin{align}
 &\text{Extrapolation operator: } \mathcal{P}_{k}(X, Y, Z) = K(Z,X)K(X, X)^{-1} \in \mathbb{R}^{},
 \\
& \text{Interpolation operator: }\mathcal{P}_{k}(X, Z, Z) = K(Z, Z)K(X, Z)^{-1}. (\#eq:EI)
\end{align}
These operators sometimes generate computational issues, due to the fact that the Kernel matrix $K(X, X) \in \mathbb{R}^{N_x \times N_x}$ must be inverted \@ref(eq:P) and this is a rather costly computational step in presence of a large set of input data. This is our motivation for introducing the additional variable $Y$ which has the effect to lower the computational cost. It reduces the overall algorithmic complexity of \@ref(eq:P) to the order of
$$
D \, \big( (N_y)^3 + (N_y)^2 N_x + (N_y)^2 N_z \big). 
$$
Most importantly, the projection operator $\mathcal{P}_{k}$ is \textit{linear} in term of, both, input and output data. Hence, while keeping the set $Y$ to a reasonable size, we can consider large set of data, as input or output. 

The reader can imagine also that choosing a relevant set $Y$ is a major source of optimization. We use this idea intensively in several applications. For instance, kernel clustering methods described in the section \@ref(a-kernel-based-clustering-algorithm) aims minimizing the error committed by our learning machine with respect to the set $Y = \mathcal{P}_{k}(X,Z)$, called **sharp discrepancy sequences** and defined below. We often refer to this step as **learning**, as this step is exactly the counterpart of the weight set for neural network approach. This construction amounts to define a feed-backward machine, analogous to \@ref(eq:P), as
$$
f_z := \mathcal{P}_{k}(X,\mathcal{P}_{k}(X,Z),Z)f(X)
$$

Note that \@ref(eq:P) allows us also to compute the following operation, where $\nabla := (\partial_1,\ldots,\partial_D)$ holds for the gradient
$$ 
 (\nabla f)(Z) := ( \nabla \mathcal{P}_{k} )(X,Y,Z)f(X):= (\nabla_z k)(Z,Y)K(X,Y)^{-1}f(X) \in \mathbb{R}^{D \times N_z \times D_f}
$$
meaning that $\nabla \mathcal{P}_{k} \in \mathbb{R}^{D \times N_z \times N_x}$ is interpreted as a tensor operator. This operator is described in the section \@ref(discrete-differential-operators), as well as numerous others, as for instance Laplacian, Leray, integral operators, that are based on it. They will be used in designing computational methods for problems involving partial differential equations(PDEs).

### Transportation maps

Let us define an important concept of a **transportation map**. A transportation map $S$ is a surjective map 
$$
S:\mathbb{R}^T \mapsto \mathbb{R}^D.
$$
There are several types of transportation maps such as

- **rescaling maps** when $T=D$, in order properly the data $X,Y,Z$ to a given kernel,
- **dimension reduction maps** when $T \leq D$, or 
- **dimension augmentation** when $T \geq D$, when adding information to the training set is required. This transformation is sometimes called a **kernel trick**.

The list of maps available in our framework is given in the table \@ref(tab:149). 

In [ ]:
maps = pd.DataFrame(factories.get_map_factory_keys()).reset_index(drop=True)
maps1 = maps[0:4]
maps2 = maps[4:9]

In [ ]:
%%R
maps1 = py$maps1
maps2 = py$maps2
rownames(maps1) <- c()
rownames(maps2) <- c()
knitr::kable(py$maps, caption = "A list of available transportation maps.", col.names = NULL)

Using a map $S$ amounts to change the kernel as $k(x,y) \mapsto k(S(x),S(y))$. 
For instance, for Gaussian kernels the map \textit{scale\_to\_min\_distance} is usually a good choice: this map scales all points to the average min distance, namely 
$$
S(x) = \frac{x}{\sqrt{\alpha}}, \qquad \alpha = \frac{1}{N} \sum_{i\le N} \min_{k \neq i}  |x^i-x^k|^2.
(\#eq:S).
$$
Let us transform our Gaussian kernel with this map. Observe that, from the signature of the Gaussian setter function, we see that the Gaussian kernel is handled with the default map $set\_min\_distance\_map$. We do not discuss all optional parameters now, but refer the reader to a later section. 

In [ ]:
map_setters.set_standard_min_map()

$$
\begin{aligned}
kernel\_setters.set\_gaussian\_kernel(& polynomial\_order:int = 0,\\
& regularization:float = 1e-8, \\
& set\_map = map\_setters.set\_min\_distance\_map)
\end{aligned}
$$

### Extrapolation, interpolation, and projection

The Python function in our framework that describes the projection operator $\mathcal{P}_{k}$ is based on the definition in \@ref(eq:P), namely 
$$
  f_z = \text{op.projection}(X,Y,Z, f(X)=[], k=None, rescale = False) \in \mathbb{R}^{N_z \times D_f}. (\#eq:opP)
$$
The optional values in this function are as follows:

- The function $f(X)$ is optional, meaning that the user can retrieve the whole matrix $\mathcal{P}_{k}(X,Y,Z) \in \mathbb{R}^{N_z \times N_x}$ if desired.
- The kernel $k$ is optional, meaning that we let to the user the freedom to re-use an already input kernel. 
- The optional value \textit{rescale}, defaulted to false, allow to call the map prior of performing the projection operation \@ref(eq:P), in order to compute its internal states for proper data scaling. For instance, a rescaling \@ref(eq:S) computes $\alpha$ according to the set ($X,Y,Z$). 

Interpolation and extrapolation Python functions are, in agreement with \@ref(eq:EI), simple decorators toward the operator $\mathcal{P}_{k}$; see \@ref(eq:opP). Obviously, the main question arising at this stage is how good the approximation is $f_z$ compared to $f(Z)$, which is the question addressed in the next section. 
$$
 f_z = \text{op.extrapolation}(X,Z,f(X) = [],\ldots), \quad f_z = \text{op.interpolation}(X,Z,f(X) = [],\ldots) (\#eq:EIP)
$$

### Functional spaces and Kolmogorov decomposition. 

Given any finite collection of points $X = [x^1 ... x^{N_x}]$, $x^i\in \mathbb{R}^{D}, i= 1,...,N_x$, we introduce a (finite dimensional) vector space $\mathcal{H}_k^x$ consisting of all linear combinations of the \textit{basis functions} $x \mapsto k(x, x^n)$. In other words, we set
\begin{equation}
\mathcal{H}_k^x = \Big\{\sum_{1 \leq m \leq N_x} a_m k(\cdot, x^m) \,  / \,  a = (a^1, \ldots, a^{N_x}) \in \mathbb{R}^{N_x}  \Big\}. (\#eq:HKx)
\end{equation} 
The \textbf{functional space} $\mathcal{H}_{k}$ is (after suitably passing to a limit in \@ref(eq:HKx)) 
\begin{equation}
\mathcal{H}_k
=  \text{Span} \big\{k(\cdot, x) \, / \, x \in \mathbb{R}^D \big\}.(\#eq:Hk)
\end{equation} 
This space consists of all linear combinations of the functions $k(x, \cdot)$ (that is, parametrized by $x \in \mathbb{R}^D$) and is endowed with the scalar product 
\begin{equation}
\big\langle k(\cdot, x), k(\cdot, y) \big\rangle_{\mathcal{H}_{k}} = k(x,y), \qquad x, y \in \mathbb{R}^D. 
\end{equation}
On every finite dimensional subspace $\mathcal{H}_{k}^x \subset \mathcal{H}_{k}$ we can check that, according to the expression of the scalar product, 
\begin{equation}
\big\langle k(\cdot, x^i), k(\cdot, x^j) \big\rangle_{\mathcal{H}_{k}^x} = k(x^i,x) K(X,X)^{-1} k(x, x^j) = k(x^i,x^j), \ i,j = 1,...,N_x.
\end{equation}

The norm of any function $f$, in view of the functional space $\mathcal{H}_{k}$, is fully determined by the kernel $k$. A reasonable approximation of this norm, which is induced by the kernel matrix $K$ is given by 
$$
  \|f\|_{\mathcal{H}_{k}}^2 \sim f(x^i)^T K(X,X)^{-1} f(x^i), \quad x^i \in \mathbb{R}^D, \ i = 1,...,N_x
$$
It is computed via the function in Python:
$$
    op.norm(X,Y,Z,f(X), set\_codpy\_kernel = None, rescale = True).
$$
Again we let the reader the choice to perform a rescaling of the kernel based on a transport map.

### Error estimates based on the discrepancy error

Recall the notation for the projection operator \@ref(eq:P). Then the following estimation error holds for any vector-valued function $f:\mathbb{R}^D \mapsto \mathbb{R}^{D_f}$:
$$
   \Big| \frac{1}{N_x}\sum_{n=1}^{N_x} f(x^n) - \frac{1}{N_z}\sum_{n=1}^{N_z} f_{z^n} \Big| \le \Big( d_k\big(X,Y\big) + d_k\big(Y,Z\big) \Big) \|f\|_{\mathcal{H}_{k}}.
$$
Before describing this formula, let us precise that it is a computationally tractable one, that can be systematically applied to check the pertinence of any kernel machine. It is also a quite general one: this formula can be adapted to others kind of error measuring. We have also 
$$
  \Big\| f(Z) - f_z \Big\|_{\ell^2(N_z)^{D_f}}\le \Big( d_k\big(X,Y\big) + d_k\big(Y,Z\big) \Big) \|f\|_{\mathcal{H}_{k}}. (\#eq:err)
$$
This error formula can be split into two parts.

The first part, $\Big( d_k\big(X,Y\big) + d_k\big(Y,Z\big) \Big)$ measures some kernel-related distance between a set of points, that we call the \textbf{discrepancy error}. It is a quite natural quantity, as one expects that the quality of an extrapolation degrades if the extrapolation set $Z$ move away from the sampling set $X$. Its definition is
$$
d_k\big(X,Y\big)^2 := \frac{1}{N_x^2}\sum_{n=1,m=1}^{N_x,N_x} k\big(x^n,x^m\big) + \frac{1}{N_y^2}\sum_{n=1,m=1}^{N_y,N_y} k\big(y^n,y^m\big) - \frac{2}{N_x N_y}\sum_{n=1,m=1}^{N_x,N_y} k\big(x^n,y^m\big) (\#eq:dk)
$$
and is described in the Python function
$$
  op.discrepancy(X,Y,Z,set\_codpy\_kernel = None, rescale = True)
$$
In particular, the user should pay attention to an undesirable rescaling effect due to the variable *rescale*. The section \@ref(a-study-of-the-discrepancy-functional) contains plots and some analysis of this functional. This distance was named Maximum Mean Discrepancy(MMD) and introduced independently in \cite{GR}.

## Dealing with kernels 

### Maps and kernels. 

**Maps can ruin your prediction**. In the figure \@ref(fig:786) We compare the ground truth values $(Z,f(Z)) \in \mathbb{R}^{N_z \times D} \times \mathbb{R}^{N_z \times D_f}$ and the predicted values  $(Z,f_z)  \in \mathbb{R}^{N_z \times D} \times \mathbb{R}^{N_z \times D_f}$ figure \@ref(fig:786).  we set a different map, called set\_mean\_distance\_map, which scales all points to the average distance for a Gaussian kernel:
\begin{equation}
S(Z) = \frac{Z}{\sqrt{\alpha}}, \qquad \alpha = \sum_{i,k \le N_z} \frac{|z^i-z^k|^2}{N_z^2}.
\end{equation}

This example illustrates how maps can drastically influence the computation.

In [ ]:
fz_extrapolated = op.extrapolation(x,fx,z,set_codpy_kernel = 
kernel_setters.set_gaussian_kernel(0,1e-8,map_setters.set_mean_distance_map),rescale = True)
multi_plot([(z,fz),(z,fz_extrapolated)],plot_trisurf,projection='3d')

However, the very same map can be appropriate for other kernels; see Figure \@ref(fig:787) with a Matern kernel. 

In [ ]:
matern_kernel = kernel_setters.set_matern_tensor_kernel(0,1e-8,map_setters.set_mean_distance_map)
fz_extrapolated = op.extrapolation(x,fx,z,set_codpy_kernel =matern_kernel,rescale = True)

In [ ]:
multi_plot([(z,fz),(z,fz_extrapolated)],plot_trisurf,projection='3d')

**Composition of maps**. Maps can be composed together. For instance, consider the following Python definition of one of our maps, which we may use as a Swiss-knife map for Gaussian kernels: 

In [ ]:
def set_standard_min_map(**kwargs):
    map_setters.set_min_distance_map(**kwargs)
    pipe_map_setters.pipe_erfinv_map()
    pipe_map_setters.pipe_unitcube_map()

For any $X \in \mathbb{R}^{N_x \times D}$, this composite map performs the following operations:

* First rescale all data to the unit cube: 

\begin{equation}
  S(X) = \frac{x_d - \min_n{x_d^n} + \frac{0.5}{N_x}}{\alpha_d}, \quad \alpha_d := \max_n{x_d^n} - \min_n{x_d^n}
\end{equation}

* Then apply the map defined as 

\begin{equation}
S(X) = erf^{-1}(2X - 1),
\end{equation}
$erf^{-1}$ being the inverse of the standard error function $erf$.

* Finally use the average min distance map, defined as 

\begin{equation}
S(X) = \frac{X}{\sqrt{\alpha}}, \quad \alpha = \frac{1}{N_x} \sum_{i\le N_x} \min_{k \neq i} |x_i-x_k|^2.
\end{equation}

### Illustration of different kernels prediction

As is clear from the previous sections, the external parameters of a kernel-based prediction machine are 

* The kernel. In most situations, a kernel is defined by
  
    * A positive definite kernel, that is one element of table \@ref(tab:249).
    * A map, that is one element of table \@ref(tab:149).
    
* The choice of the inner parameters set $Y$. We usually face three main choices here.

    * $Y=X$, that corresponds to the *extrapolation* case, section \@ref(extrapolation-interpolation-and-projection). This is the most efficient choice when one seeks for high accuracy results.
    * $Y$ is randomly picked among $X$. This choice trades accuracy for execution time and is adapted to heavy training set.
    * $Y$ is set as a sharp discrepancy sequence of $X$, described in the section \@ref(a-kernel-based-clustering-algorithm). This choice optimizes accuracy versus execution time. These are the most possible accurate machine at a given computational burden but involves a time-consuming algorithm.

To illustrate the effects of different kernels and maps on learning machines, we compare in this section predictions for the one-dimensional test described the section \@ref(periodic-kernel-for-machine-learning), for different kernels.

<!-- ```{python} -->
<!-- data_random_generator_ = data_random_generator(fun = my_fun,types=["cart","sto","cart"]) -->
<!-- data_accumulator_ = data_accumulator() -->
<!-- scenarios_list = [ (1, 100, 100 ,100 ) ] -->
<!-- scenario_generator_ = scenario_generator() -->
<!-- scenario_generator_.run_scenarios(scenarios_list,data_random_generator_, -->
<!-- codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_gaussianper_kernel,2,1e-8,None)), -->
<!-- data_accumulator_) -->
<!-- scenario_generator_.run_scenarios(scenarios_list,data_random_generator_, -->
<!-- codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_gaussianper_kernel,0,1e-8,None)), -->
<!-- data_accumulator_) -->
<!-- scenario_generator_.run_scenarios(scenarios_list,data_random_generator_, -->
<!-- codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_gaussian_kernel,0,1e-8,map_setters.set_standard_mean_map)), -->
<!-- data_accumulator_) -->
<!-- scenario_generator_.run_scenarios(scenarios_list,data_random_generator_, -->
<!-- codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_gaussian_kernel,0,1e-8,map_setters.set_standard_min_map)), -->
<!-- data_accumulator_) -->
<!-- scenario_generator_.run_scenarios(scenarios_list,data_random_generator_, -->
<!-- codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_matern_tensor_kernel,0,1e-8,None)), -->
<!-- data_accumulator_) -->
<!-- scenario_generator_.run_scenarios(scenarios_list,data_random_generator_, -->
<!-- codpyexRegressor(set_kernel = kernel_setters.set_linear_regressor_kernel), -->
<!-- data_accumulator_) -->
<!-- list_results = [(a,b) for a,b in zip(scenario_generator_.accumulator.get_zs(),scenario_generator_.accumulator.get_f_zs())] -->
<!-- multi_plot(list_results,plot1D,mp_max_items = -1,f_names=["piped: linear and periodical gaussian kernel, no map","periodical gaussian kernel, no map","gaussian kernel with mean distance map","gaussian kernel with min map","matern kernel, no map","linear regressor kernel, no map"]) -->
<!-- ``` -->




In [ ]:
data_random_generator_ = data_random_generator(fun = my_fun,types=["cart","sto","cart"])
data_accumulator_ = data_accumulator()
scenarios_list = [ (1, 100, 100 ,100 ) ]
scenario_generator_ = scenario_generator()
scenario_generator_.run_scenarios(scenarios_list,data_random_generator_,
codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_gaussianper_kernel,2,1e-8,None)), data_accumulator_)

scenario_generator_.run_scenarios(scenarios_list,data_random_generator_,
codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_gaussianper_kernel,0,1e-8,None)),data_accumulator_)
list_results = [(a,b) for a,b in zip(scenario_generator_.accumulator.get_zs(),scenario_generator_.accumulator.get_f_zs())]
multi_plot(list_results,plot1D,mp_max_items = -1,f_names=["piped: linear and periodical gaussian kernel, no map", "periodic gaussian kernel, no map"])

In [ ]:
scenario_generator_ = scenario_generator()
data_accumulator_ = data_accumulator()
scenario_generator_.run_scenarios(scenarios_list,data_random_generator_,
codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_gaussian_kernel,0,1e-8,map_setters.set_standard_mean_map)),data_accumulator_)
scenario_generator_.run_scenarios(scenarios_list,data_random_generator_,
codpyexRegressor(set_kernel = kernel_setters.set_linear_regressor_kernel), data_accumulator_)
list_results = [(a,b) for a,b in zip(scenario_generator_.accumulator.get_zs(),scenario_generator_.accumulator.get_f_zs())]
multi_plot(list_results,plot1D,mp_max_items = -1,f_names=["gaussian kernel with mean distance map","gaussian kernel with min map"])

In [ ]:
scenario_generator_ = scenario_generator()
data_accumulator_ = data_accumulator()
scenario_generator_.run_scenarios(scenarios_list,data_random_generator_,
codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_gaussian_kernel,0,1e-8,map_setters.set_standard_min_map)),data_accumulator_)
scenario_generator_.run_scenarios(scenarios_list,data_random_generator_,
codpyexRegressor(set_kernel = kernel_setters.kernel_helper(kernel_setters.set_matern_tensor_kernel,0,1e-8,None)), data_accumulator_)
list_results = [(a,b) for a,b in zip(scenario_generator_.accumulator.get_zs(),scenario_generator_.accumulator.get_f_zs())]
multi_plot(list_results,plot1D,mp_max_items = -1,f_names=["matern kernel, no map", "linear regressor kernel, no map"])

## Kernel engineering

In this section we describe some general operations on kernels, which allow us to generate new and relevant kernels. These operations preserve a positivity property required for kernels. For this section, two kernels denoted by $k_i(x,y) : \mathbb{R}^D \times \mathbb{R}^D \mapsto \mathbb{R}, i=1,2$ are given with corresponding matrices $K_1$ and $K_2$. In agreement with \@ref(eq:P), we introduce the corresponding two projection operators:
\begin{equation} 
  \mathcal{P}_{k_i}(X,Y,Z):= K_i(Z,Y)K_i(X,Y)^{-1} \in \mathbb{R}^{N_z \times N_x}, i = 1,2 (\#eq:Pi)
\end{equation}

### Manipulating kernels
In order to work with two (or more) kernels, we introduced the following Python function, which are basic *setters* and *getters* to kernels: *get\_kernel\_ptr()* and *set\_kernel\_ptr(kernel\_ptr)*. The first one allows us to recover an already input kernel of our library, while the second one allows us to input it into our framework.

### Adding kernels

The first operation, denoted by \(k_1 + k_2\) and defined from any two kernels, consists in simply adding two kernels. If $K_1$ and $K_2$ are two kernel matrices associated to the kernels $k_1$ and $k_2$, then we define the sum of two kernels as $K(X,Y)\in \mathbb{R}^{N_x \times N_y}$ and corresponding projection operator as $\mathcal{P}_{k}(X,Y,Z) \in \mathbb{R}^{N_z \times N_y}$:
\begin{equation}
K(X,Y):=K_1(X,Y)+K_2(X,Y), \quad \mathcal{P}_{k}(X,Y,Z) = K(Z, X)K(X,Y)^{-1}. (\#eq:add)
\end{equation}

The functional space generated by \(k_1+k_2\) is then  
\begin{equation}
\mathcal{H}_k = \Big\{\sum_{1 \leq m \leq N_x} a^m (k_1(\cdot, x^m)+k_2(\cdot, x^m))\Big\}. 
\end{equation}

### Multiplication kernels

A second operation, denoted by $k_1 \cdot k_2$ and defined from any two kernels, consists in multiplying two kernels together. A kernel matrix $K(X,Y)\in \mathbb{R}^{N_x \times N_y}$ and a projection operator $\mathcal{P}_{k}(X,Y,Z) \in \mathbb{R}^{N_z \times N_y}$ corresponding to the product of two kernels are defined as
\begin{equation}
K(X,Y):=K_1(X,Y) \circ K_2(X,Y), \quad \mathcal{P}_{k}(X,Y,Z) = K(Z,X)K(X,Y)^{-1},  (\#eq:mul)
\end{equation}
where $\circ$ is the Hadamard product of two matrices. The functional space generated by \(k_1 \cdot k_2\) is
\begin{equation}
\mathcal{H}_k = \Big\{\sum_{1 \leq m \leq N_x} a^m (k_1(\cdot, x^m)k_2(\cdot, x^m))\Big\}.
\end{equation}

### Convolution kernels

Our next operation, denoted by \(k_1 \ast k_2\) and defined from any two kernels, consists in multiplying corresponding kernel matrices $K_1$ and $K_2$ as 
\begin{equation}
K(X,Y):=K_1(X,Y) K_2(Y,Y), (\#eq:ast)
\end{equation}
where $K_1(X,Y) K_2(Y,Y)$ stands for the standard matrix multiplication. The projection operator is given by $\mathcal{P}_{k}(X,Y,Z) = K(Z,X)K(X,Y)^{-1}$. Suppose that $k_1(x,y) = \varphi_1(x-y)$, $k_2(x,y) = \varphi_2(x-y)$, then
the functional space generated by \(k_1 \ast k_2\) is 
\begin{equation}
\mathcal{H}_k = \Big\{\sum_{1 \leq m \leq N_x} a^m (k(\cdot, x^m))\Big\}, 
\end{equation}
where $k(x,y) := \Big( \varphi_1 \ast \varphi_2 \Big)(x-y)$ is the convolution of the two kernels.

### Piped kernels

Another important operation is referred to here as "piping kernels" and provides yet another route for generating new kernels in a natural and explicit way.  It is denoted by \(k_1 | k_2\) and corresponds to define the projection operator \@ref(eq:opP) as follows: 
\begin{equation}
  \mathcal{P}_{k}(X,Y,Z) = \mathcal{P}_{k_1}(X,Y,Z) \pi_1(X,Y) + \mathcal{P}_{k_2}(X,Y,Z)\Big( I_d-\pi_1(X,Y)\Big), (\#eq:pipe)
\end{equation} 
where the projection operator here reads 
\[
 \pi_1(X,Y):=K_1(X,Y)K_1(X,Y)^{-1} = \mathcal{P}_{k_1}(X, Y, X).
\]
This operation splits the projection operator $\mathcal{P}_{k}(X,Y,Z)$ into two parts. The first part is handled by a single kernel, while the second kernel handles the remaining error. From a mathematical standpoint point, this is equivalent to a functional Gram-Schmidt orthogonalization process of both functional spaces $\mathcal{H}_{k_1}^x$, $\mathcal{H}_{k_2}^x$, and the corresponding functional space defined by \@ref(eq:pipe) is, after \@ref(eq:Hk), 
\begin{equation}
\mathcal{H}_k^x = \Big\{\sum_{1 \leq m \leq N_x} a^m k_1(\cdot, x^m) + \sum_{1 \leq m \leq N_x} b^m k_2(\cdot, x^m)  \Big\}. 
\end{equation}
Hence, this doubles up the coefficients \@ref(eq:coefy). We define its inverse concatenating matrix 
\begin{equation}
  K^{-1}(X,Y) = \Big( K_1(X,Y)^{-1} , K_2(X,Y)^{-1}\big( I_{N_x}-\pi_1(X,Y)\big)\Big) \in \mathbb{R}^{2N_y \times N_x}.
\end{equation}
The kernel matrix associated to a "piped" kernel pair is then
\begin{equation}
  K(X,Y) = \Big(K_1(X,Y) , K_2(X,Y) \Big) \in \mathbb{R}^{N_x \times 2 N_y}. 
\end{equation}

### Piping scalar product kernels: an example with a polynomial regression

Let $S : \mathbb{R}^D \mapsto \mathbb{R}^N$ be given by a family of $N$ basis functions $\varphi_n$, i.e. $S(x) = \Big(\varphi_1(x),\ldots,\varphi_N(x)\Big)$ and consider the following kernel, called dot product kernel (which is conditionally positive-definite): 
\begin{equation}
	k_1(x,y):= <S(x),S(y)>. (\#eq:LRK)
\end{equation}
Now, given any positive kernel $k_2(x,y)$, consider a "pipe" kernel $k_1 | k_2$. In particular, such a construction is very useful with a polynomial basis function $S(x) = \Big(1,x_1,\ldots\Big)$ : it consists of a classical polynomial regression, allowing to perfectly match moments of distributions, since the remaining error is handled by the second kernel.

### Neural networks viewed as kernel methods.

Our setup describes the strategies developed in deep learning theory, which are based on neural networks. Namely, we consider any feed-forward neural network of depth $M$, defined by  
$$
z_m = y_{m} g_{m-1}(z_{m-1})\in \mathbb{R}^{N_{m} }, \qquad y_m \in \mathbb{R}^{N_{m} \times N_{m-1}}, 
\qquad z_0 = y_0 \in \mathbb{R}^{N_0}, 
$$
in which $y_0,\ldots,y_M$ are considered as weights and $g_{m}$ as prescribed activation functions. By concatenation, we arrive at the function 
$$
z_M(y) = y_M z_{M-1}(y_0, \ldots,y_{M-1}) : \mathbb{R}^{N_0 \times \ldots \times N_M} \mapsto \mathbb{R}^{N_M }.
$$
This neural network is thus entirely represented by the kernel composition
$$
k(y_m,\ldots,y_0) = k_m \Big(y_m, k_{m-1} \big( \ldots, k_1(y_1,y_0) \big) \Big) \in \mathbb{R}^{ N_m \times \ldots \times N_0}
$$ 
where $k_m(x,y) = g_{m-1}(x y^T)$, indeed $z_M(y) = y_M k(y_{M-1},\ldots,y_0)$. 

## Discrete differential operators 

### Coefficient operator
We start this section by further analyzing the projection operator \@ref(eq:P). We can interpret this operator in a basis function setting:
\begin{equation} 
 f_z := K(Z,Y) c_y, \quad c_y:= K(X,Y)^{-1}f(X    ) \in \mathbb{R}^{N_y \times D_f}. (\#eq:coefy)
\end{equation}
The coefficients $c_y$ corresponds to the representation of $f$ in a basis of functions 
\begin{equation} 
 f_z := \sum_{n=1}^{N_y} c_y^n K(Z,y^n), 
\end{equation}
Coefficients are matrices also, having size $N_y \times D_f$, except for some composite kernels. The table \@ref(tab:289) shows the first four coefficients of the test function $f_z$.

In [ ]:
coef = op.coefficients(x,y,fx,set_codpy_kernel=set_kernel,rescale=True)
#print("coefficient shape:",coef.shape)

In [ ]:
%%R
knitr::kable(py$coef[0:4], caption = "First four rows coefficients matrix", col.names = NULL)

### Partition of unity

For any $Y \in \mathbb{R}^{N_y \times D}$, consider the projection operator \@ref(eq:P), and the following vector-valued function: 
\begin{equation} 
 \phi : Y \mapsto \Big(\phi^1(Y),\ldots,\phi^{N_x}(Y) \Big) = K(Y,X)K(X,X)^{-1} \in \mathbb{R}^{N_y \times N_x}.(\#eq:PU) 
\end{equation}
that corresponds to the projection operator $\mathcal{P}_k(X,X,Y)$. On every point $x^n$, one computes (without considering regularization terms)
\begin{equation} 
 \phi(x^n):= \Big(0,\ldots,1,\ldots,0\Big) = \delta_{n,m}, 
\end{equation}
where $\delta_{n,m} = 1$ if $n=m$, $0$ else. For this reason, we call $\phi(x)$ a partition of unity. The figure \@ref(fig:part) illustrates partitions functions. 

<!-- The following code allows to compute this function, using straightforwardly a previous section:  -->

In [ ]:
partxxz = op.projection(x,x,z).T

<!-- and we plot some of these functions in the figure \@ref(fig:part):  -->

In [ ]:
temp = int(np.sqrt(Nz))
multi_plot([(z,partxxz[0,:]),(z,partxxz[int(temp/3),:])],plot_trisurf,projection='3d',elev = 30)

In [ ]:
temp = int(np.sqrt(Nz))
multi_plot([(z,partxxz[int(temp*2/3),:]),(z,partxxz[temp-1,:])],plot_trisurf,projection='3d',elev = 30)

### Gradient operator

For any positive-definite kernel $k$, and points $X, Y, Z$, we define $\nabla$ operator as the 3-tensor: 
$$
  \nabla_{k}(X,Y,Z) = \Big(\nabla_z k\Big)(Z,Y)K(X,Y)^{-1} \in \mathbb{R}^{D \times N_x\times N_z}, 
$$
where $\Big(\nabla_z k\Big)(Z,Y) \in \mathbb{R}^{D \times N_x\times N_y}$ is interpreted as a three-tensor. The gradient of any vector valued function $f$, is computed as 
$$( \nabla f)(Z) \sim (\nabla_{k})(Z,Y,Z) f(X) \in \mathbb{R}^{D \times N_z\times D_f},$$

where we omit the dependence $\nabla_{k}(X,Y,Z)$ for concision. Observe that maps, introduced in the section \@ref(transportation-maps), modify the operator $\nabla_{k}$  as follows: 
\begin{equation}
 \nabla_{k\circ S} (X, Y, Z)  := (\nabla S)(Z)\Big(\nabla_1 k)(S(Z),S(Y) \Big) K(S(X),S(Y))^{-1}, (\#eq:nablak)
\end{equation}
where $\Big(\nabla_1 k\Big)(Z,Y) \in \mathbb{R}^{D \times N_z\times N_y}$ is interpreted as a three-tensor, as is $(\nabla S)(Z) := (\partial_d S^j)(Z^{n_z}) \in \mathbb{R}^{D \times D \times N_z}$, representing the Jacobian of the map $S$, and the multiplication holds for the two first indices.

\textbf{Two-dimensional example.} First we compare the original and extrapolated functions in the figure \@ref(fig:nablacp): 

In [ ]:
fz_extrapolated = op.extrapolation(x, fx, z, set_codpy_kernel=set_kernel, rescale = True)
multi_plot([(z, fz),(z,fz_extrapolated)],plot_trisurf,projection='3d')

The figures \@ref(fig:nabla1) and \@ref(fig:nabla2) illustrate a corresponding derivative and compare it to the original one for the first and second dimensions respectively.

In [ ]:
nabla_f_x = op.nabla(x=x,y=x,z=z,fx=fx)
multi_plot([(z,nabla_fz[:,0,:]),(z,nabla_f_x[:,0,:])],plot_trisurf,projection='3d')

In [ ]:
nabla_f_x = op.nabla(x=x,y=x,z=z,fx=fx)
multi_plot([(z,nabla_fz[:,1,:]),(z,nabla_f_x[:,1,:])],plot_trisurf,projection='3d')

### Divergence operator

We define the $\nabla^T$ operator as the transpose of the 3-tensor operator $\nabla$: 
$$
  <\nabla_{k}(X,Y,Z)f(X),g(Z)> = <f(X),\nabla_{k}(X,Y,Z)^T g(Z)>. 
$$
Hence, as the left-hand side is homogeneous with, for any smooth function $f$ and vector fields $g$, and $\nabla \cdot$ denotes the divergence operator
\begin{equation}
  \int (\nabla f) \cdot g d\mu = -\int f \nabla \cdot (g d\mu). (\#eq:578)
\end{equation}
The $\nabla^T$ operator is thus consistent with the divergence operator
$$
  \nabla_{k}(X,Y,Z)^T f(Z) \sim - \nabla \cdot (f\mu)(x)
$$
To compute this operator, we proceed as follows: starting from the definition of the gradient operator \@ref(eq:nablak), we define, for any $f(X) \in \mathbb{R}^{N_x \times D_f}$,$g(Z) \in \mathbb{R}^{D \times N_z \times D_f}$
$$
  <\Big(\nabla_z K\Big)(Z,Y)K(X,Y)^{-1}f_x,g_z> = <f_x,K(X,Y)^{-T}\Big(\nabla_z K\Big)(Z,Y)^T g_z>
$$
Thus the operator $\nabla_{k}(X,Y,Z)$ is defined as: 
$$
  \nabla_{k}(X,Y,Z)^T = K(X,Y)^{-T} \Big(\nabla_z K\Big)(Z,Y)^T \in \mathbb{R}^{N_x \times N_z D}, 
$$
where $\nabla_z K(Z,Y)^T \in \mathbb{R}^{N_y \times (N_z D)}$ is the transpose of the matrix $\nabla_z K(Z,Y)$. 

\textbf{A two-dimensional example.}

In the figure \@ref(fig:nablaTnabla) we illustrate the material in this section by comparing $\nabla_{k}(X,Y,Z)^T \nabla_{k}(X,Y,Z) f(X)$ to $\Delta_{k}(X,Y) f(X)$; see the next section. 

In [ ]:
f1 = op.nablaT(x,y,x,op.nabla(x,y,x,fx))
f2 = op.nablaT_nabla(x,y,fx)
multi_plot([(x,f1),(x,f2)],plot_trisurf,projection='3d')

### Laplace operator 

We define the Laplace operator as the matrix
$$
  \Delta_{k}(X,Y) = \Big(\nabla_{k}(X,Y,X)^{T}\Big)\Big(\nabla_{k}(X,Y,X)\Big) \in \mathbb{R}^{ N_x \times N_x}.
$$
This operator is not consistent with the "true" Laplace operator, but is instead consistent with   \@ref(eq:578)
$$
  -\nabla \cdot (\nabla f\mu).
$$

Illustration for this section is done in the figure \@ref(fig:nablaTnabla).

### Inverse Laplace operator

This operator is simply the pseudo-inverse of the Laplacian $\Delta_{k}(X,Y) \in \mathbb{R}^{N_x \times N_x}$.

\textbf{A two-dimensional example.}

Figure \@ref(fig:Delta1) compares $f(X)$ with $\Delta_{k}(X,Y)^{-1}\Delta_{k}(X,Y)f(X)$. This latter operator is a projection operator (hence is stable).

In [ ]:
temp = op.nablaT_nabla(x,y,op.nablaT_nabla_inv(x,y,fx))
multi_plot([(x,fx),(x,temp)],plot_trisurf,projection='3d')

We also compute the operator $\Delta_{k, x, y, z}\Delta_{k, x, y, z}^{-1}f(X)$ figure \@ref(fig:Delta2), to check that pseudo-inversion commutes, as highlighted by the following computations: 

In [ ]:
temp = op.nablaT_nabla_inv(x,y,fx)
temp = op.nablaT_nabla(x,y,temp)
multi_plot([(x,fx),(x,temp)],plot_trisurf,projection='3d')

### Integral operator - inverse gradient operator

The following operator $\nabla^{-1}_k$ is an integral-type operator
$$
  \nabla^{-1}_{k} =  \Delta_{k}^{-1}\nabla_{k}^T \in \mathbb{R}^{ N_x \times DN_z}. 
$$
It can be interpreted as a matrix, computed first considering $\nabla_{k}(X,Y,Z)\in \mathbb{R}^{D \times N_z\times N_x}$, down casting it to a matrix $\mathbb{R}^{ DN_z \times N_x}$ before performing a least-square inversion. This operator acts on any 3-tensor $v_z \in \mathbb{R}^{ D \times N_z \times D_{v_z}}$, and outputs a matrix
$$
  \nabla^{-1}_{k}(X, Y, Z) v_z \in \mathbb{R}^{ N_x \times D_{v_z}}, \quad v_z \in \mathbb{R}^{ D \times N_z \times D_{v_z}}
$$
The operator $\nabla^{-1}_{k}$ corresponds to the minimization procedure: 
$$
  h(X) := \arg \inf_{h \in \mathbb{R}^{ N_x \times D_{v_z}}}\| \nabla_{k}(X,Y,Z) h - v_z \|_{D,N_z,N_x}^2.
$$

\textbf{A two-dimensional example.}

In the figure \@ref(fig:NablainvNabla) we show that $(\nabla )(\nabla )^{-1}f(X)$ coincides with $f(X)$. Observe however that this latter operation is not equivalent to Figure \@ref(fig:NablainvNabla2), which uses $Z$ as extrapolation set. 

In [ ]:
fz_inv = op.nabla_inv(x,y,x,op.nabla(x,y,x,fx))
multi_plot([(x,fx),(x,fz_inv)],plot_trisurf,projection='3d')

In [ ]:
fz_inv = op.nabla_inv(x,y,z,op.nabla(x,y,z,fx))
multi_plot([(x,fx),(x,fz_inv)],plot_trisurf,projection='3d')

### Integral operator - inverse divergence operator

The following operator $(\nabla_k^T)^{-1}$ is another integral-type operator of interest.
We define the $(\nabla^T)^{-1}$ operator as the pseudo-inverse of the $\nabla^T$ operator:
$$
  (\nabla^T_{k}(X,Y,Z))^{-1}= \nabla_{k}(X,Y,Z)\Delta_{k}(X,Y,Z)^{-1}.
$$

\textbf{A two-dimensional example.}

We compute $\nabla_{k}(X,Y,Z)^T(\nabla^T_{k}(X,Y,Z))^{-1}= \Delta_{k}(X,Y,Z)\Delta_{k}(X,Y,Z)^{-1}$. Thus, the following computations should give comparable results as those obtained in the section concerning the inverse Laplace operator; see section \@ref(inverse-laplace-operator).

In [ ]:
temp = op.nablaT(x,y,x,op.nablaT_inv(x,y,x,fx))
multi_plot([(x,fx),(x,temp)],plot_trisurf,projection='3d',elev = 30)

### Leray-orthogonal operator

We define the Leray-orthogonal operator as 
$$
  L_{k}(X,Y)^\perp := \nabla_{k}(X,Y) \Delta_{k}(X,Y)^{-1} \nabla_{k,x,y,x}^T = \nabla_{k}(X,Y,Z) \nabla_{k}(X,Y,Z)^{-1}. 
$$
This operator acts on any vector field $f(Z) \in \mathbb{R}^{ D \times N_z \times D_f}$, down casting it, performing a matrix multiplication and producing a three-tensor: 
$$
  L_{k}(X,Y,Z)^\perp f(Z) \in \mathbb{R}^{ D \times N_z \times D_f}.
$$
In the figure \@ref(fig:LerayT), we compare this operator to the original function $(\nabla f)(Z)$: 

In [ ]:
#LerayT_fz = op.Leray_T(z,y,nabla_fz)
LerayT_fz = op.nabla(x,y,z,op.nabla_inv(x,y,z,nabla_fz))
multi_plot([(z,nabla_fz[:,0,:]),(z,LerayT_fz[:,0,:]),(z,nabla_fz[:,1,:]),(z,LerayT_fz[:,1,:])],plot_trisurf,projection='3d')

### Leray operator and Helmholtz-Hodge decomposition

We define the Leray operator as 
$$
  L_{k}(X,Y,Z) := I_d-L_{k}(X,Y,Z)^\perp = I_d - \nabla_{k}(X,Y,Z) \Delta_{k}(X,Y,Z)^{-1} \nabla_{k}(X,Y,Z)^T, 
$$
where $I_d$ is the identity. Hence, we get the following orthogonal decomposition of any tensor fields:
$$
  v_z = L_{k}(X,Y,Z) v_z + L_{k}(X,Y,Z)^\perp v_z, \quad <L_{k}(X,Y,Z) v_z,L_{k}(X,Y,Z)^\perp v_z>_{D,N_z,D_v} = 0.
$$
This agrees with the Helmholtz-Hodge decomposition, decomposing any vector field into an orthogonal sum of a gradient and a divergence free vector:
$$
  v = \nabla h + \zeta, \quad \nabla \cdot \zeta = 0, \quad h := \Delta^{-1} \nabla \cdot v 
$$
Here we have also an orthogonal decomposition from a numerical point of view:
$$
  v_z = \nabla_{k}(X,Y,Z) h_x + \zeta_z, \quad h_x := \nabla_{k}(X,Y,Z)^{-1} v_z, \quad \zeta_z := L_{k}(X,Y,Z) v_z, 
$$
satisfying numerically
$$
  \nabla_{k}(X,Y,Z)^T \zeta_z = 0, \quad \left<\zeta_z, \nabla_{k}(X,Y,Z) h_x\right>_{D \times N_z \times D_f} = 0.
$$
In the following figure, we compare this operator to the original function $(\nabla f)(Z)$ in the figure \@ref(fig:Leray).

In [ ]:
Leray_fz = nabla_fz - LerayT_fz
multi_plot([(z,nabla_fz[:,0,:]),(z,Leray_fz[:,0,:]),(z,nabla_fz[:,1,:]),(z,Leray_fz[:,1,:])],plot_trisurf, projection='3d')

## A kernel-based clustering algorithm

In this section we describe a kernel-based clustering algorithm. This algorithm, already presented with a toy example in the section \@ref(benchmark-methodology-for-unsupervised-learning), is also benchmarked in a forthcoming section devoted to more concrete problems, see Chapter \@ref(application-for-unsupervised-machine-learning).

### Distance-based unsupervised learning machines

Distance-based minimization algorithms can be thought as finding a minimum of a distance between set of points $d_k(X, Y)$, defining equivalently a distance between discrete measures $\mu_x$, $\mu_y$. Within this setting, minimization problem can be expressed as
$$
Y = \arg \inf_{Y \in \mathbb{R}^{N_y \times D}} d_k(X,Y)  (\#eq:dist)
$$

Suppose that this last problem is well-posed, assuming that the distance functional is convex\footnote{although most of existing distance are not convex}. Once the cluster set $Y:=\Big(y^1,\ldots,y^{N_y} \big)$ is computed, then one can define the index function $\sigma_d(w, Y) := \{ j : d(w, y^j) = \inf_k d(w, y^k) \}$, as for \@ref(eq:sigmaw). One we can extend naturally this function, defining a map
\begin{equation}
\sigma_d(Z,Y) := \{\sigma_d(z^1,Y), \ldots, \sigma_d(z^{N_z},Y)\} \in [1,\ldots,N_y]^{N_z}, (\#eq:sigma)
\end{equation}
that acts on the indices of the test set $Z$. This allows to compare this prediction to a given, user-desired, partition of $f(Z)$, if needed. 

Note that the function $\sigma_d(Z, Y)$ is surjective (many-to-one). Hence we can define its injective (one-to-many) inverse, $\sigma_d(Z, Y)^{-1}(n)$, describing those points of the test set attached to one $y^n$. This construction defines cells, very similarly to quantization, $C^n := \sigma_d(\mathbb{R}^D, y^n)^{-1}(n)$, defining a partition of unity of the space $\mathbb{R}^D$.
A last remark: consider, in the context of supervised clustering methods, the training set and its values $X, f(X)$ and the index map $\sigma_d(X, Y) \in [1, \ldots, N_x]^{N_y}$ defined above. One can always define a prediction on the test set $Z$ as
$$f_z := f\big(X^{\sigma(Y^{\sigma(Z, Y)},X )}\big),$$ 
showing that distance minimization unsupervised algorithm naturally defines supervised ones.

### Sharp discrepancy sequences 

Our kernel-based algorithm for clustering can be described as follows:

* The unsupervised algorithm aims to solve the minimization problem \@ref(eq:dist) with the discrepancy functional \@ref(eq:dk). 
This procedure is separated into two main steps.
  * First solve a discrete version of \@ref(eq:dist), namely
$$
X^\sigma = \arg \inf_{\sigma \in \Sigma } d(X, X^\sigma), 
$$
where $\Sigma$ denotes the set of all subsets from $[1,\ldots,N_y] \mapsto [1,\ldots,N_x]$. This minimization problem is described Chapter \@ref(general-cost-functions-and-motivations).
  * Depending on kernels, this step is completed by a simple gradient descent algorithm. The initial state for this minimization is chosen to be $X^\sigma$.
  * The resulting solution $Y$ is named **sharp discrepancy sequences**.
* The supervised algorithm consists then simply to compute the projection operator \@ref(eq:P), that we recall here.
$$
 f_z := \mathcal{P}_{k}(X,Y,Z)f(X)
$$
using the python function \@ref(eq:opP), where the *weight set* $Y$ is taken as the sharp discrepancy sequence computed above.

### Python functions

* The unsupervised clustering algorithm is given by the Python function
$$
  sharp\_discrepancy(X, Y = [], N_y =0, set\_codpy\_kernel = None, rescale = False,nmax=10)
$$
* Let $X \in \mathbb{R}^{N_x\times D}$, $Y \in \mathbb{R}^{N_y\times D}$ any two distributions of points and $k(x,y)$ a positive-definite kernel. The following Python function
$$
  codpy.alg.match(Y,X,nmax = 10)
$$
approximate the following problem
$$
  \arg \inf_{Y \in \mathbb{R}^{N_y\times D}} d_k(X,Y)^2
$$
via a simple descent algorithm: starting from the input distribution $Y$, the algorithm performs $nmax$ steps of a descent algorithm and output the resulting distribution.
* The computation of index associations \@ref(eq:sigma), that is the function $\sigma_{d_k}(X, Y)$, is given by
$$
alg.distance\_labelling(X, Y, set\_codpy\_kernel = None, rescale = True).
$$
This function relies on the distance matrix $D(X,Y)$; see \@ref(fundamental-notions-for-supervised-learning).

### Impact of sharp discrepancy sequences on discrepancy errors

The figure \@ref(benchmark-methodology-for-unsupervised-learning) presented a first illustration of the impact of computing discrepancy errors on several toy "blob" examples. In this paragraph, we fix the number of "blobs" to two, and the number of generated points $N_x$ to 100. We then follow the test methodology of the section \@ref(benchmark-methodology-for-unsupervised-learning), re-running all tests with scenarios for $N_y$ covering [0,100]. The figure \@ref(fig:741) compare the results for discrepancy errors of the three methods. One can check visually that discrepancy errors is zero, whatever the clustering method is, when the number of clusters $N_y$ tends to $N_x$. Note also that our kernel clustering methods shows quite good inertia performance indicators. This is surprising, as our method is based on discrepancy error minimization, not inertia. An interpretation could be that the inertia functional is bounded by the discrepancy error one.

In [ ]:
from clusteringCHK import *
set_kernel = set_gaussian_kernel
scenarios_list = [ (2, 500, i,500 ) for i in np.arange(10,100,5)]
validator_compute=['discrepancy_error','inertia']
scenario_generator_ = scenario_generator()
scenario_generator_.run_scenarios(scenarios_list,data_blob_generator(blob_nb=5),scikitClusterClassifier(set_kernel = set_kernel),cluster_accumulator(),validator_compute=validator_compute)
scenario_generator_.run_scenarios(scenarios_list,data_blob_generator(blob_nb=5),codpyClusterClassifier(set_kernel = set_kernel),cluster_accumulator(),validator_compute=validator_compute)
scenario_generator_.run_scenarios(scenarios_list,data_blob_generator(blob_nb=5),MinibatchClusterClassifier(set_kernel = set_kernel),cluster_accumulator(),validator_compute=validator_compute)

In [ ]:
scenario_generator_.compare_plots(axis_field_labels = [("Ny","discrepancy_errors"),("Ny","inertia")])

### A study of the discrepancy functional

As stated in the previous section, we first compute a discrete minimizing problem and denote $X^\sigma$ its solution. We eventually complete this step with a simple gradient descent algorithm. This section explains and motivate this choice. Indeed, the minimizing properties $d_k(X, Y)$ relies heavily on the kernel definition $k(x, y)$, and we face an alternative, depending on regularity of kernels, that we illustrate numerically in this section:

* If the kernel is smooth, then the distance functional $d_k(X, Y)$ also is, and a descent algorithm based on gradients computations is an efficient option.
* If the kernel is only continuous, or piecewise derivable, then we assume that the minimum is attained by the discrete minimum solution $X^\sigma$.

Hence, we study in this section the effect of some classical kernel over this functional for a better understanding.
To that aim, let us produce some random distributions $x \in \mathbb{R}^{N_x}$ in one dimension, we will study then for three kernels the following functionals:
$$
  \mathbb{R}^{N_y} \ni y \mapsto d_k(x, y), 
$$
$$
  \mathbb{R}^{N_y \times 2} \ni Y=(y^1, y^2) \mapsto d_k(x,Y).
$$

We generate uniform random variables $x \in \mathbb{R}^{N_x},y\in \mathbb{R}^{N_y}$ and $Y = (y^1, y^2) \in \mathbb{R}^{N_y \times 2}$.

In [ ]:
D,Nx,Ny,Nz=2,1000,500,2
x,y,z,Nx,Ny,Nz =  data_random_generator().get_raw_data(D=1,Nx=5,Ny=1000,Nz=0)
(y2,dummy,dummy,Nx2,dummy,dummy) = data_random_generator().get_raw_data(D=2,Nx=1000,Ny=0,Nz=0)

**A example of smooth kernels: Gaussian**. We start our study of the discrepancy functional with a Gaussian kernel. 
The Gaussian kernels is a family of kernels based upon the following kernel, generating functional spaces of smooth functions.
$$
k(x,y) = \exp(-(x-y)^2)
$$
The following picture plot the function $y \mapsto d_k(x,y)$ in blue. We also output the function $d_k(x,x^n)$, $n=1 \ldots N_x$, figure \@ref(fig:dgaussian) to illustrate that this functional is neither convex nor concave.

In [ ]:
DF = op.discrepancy_functional(x=x,y=z,set_codpy_kernel = kernel_setters.set_gaussian_kernel)
dys = DF.eval(y.reshape(y.shape[0],1,y.shape[1]))
dxs = DF.eval(x.reshape(x.shape[0],1,x.shape[1]))
compare_plot_lists([y,x],[dys,dxs])

We see that the functional $y \mapsto d_k(x,y)$ admits a minimum close to $y=\frac{1}{2}$ as expected. The next picture \@ref(fig:dgaussian2) plots $y:=(y^1,y^2) \mapsto d_k(x,y)$ 

In [ ]:
dzs = DF.eval(y2.reshape(y2.shape[0],2,int(y2.shape[1] / 2)))
multi_plot([(y2,dzs)],plot_trisurf,projection='3d',elev=30)

We see that this functional admits two minima. This reflects the fact that the functional $y \mapsto d_k(x,y)$ is invariant by permutation of the indices of $y$.

**An example of Lipschitz continuous kernels: RELU**. Let us now study a kernel generating functional spaces having less regularity. The norm kernels is a family of kernels based upon the following kernel, generating functional spaces of bounded variation functions.
$$
k(x, y) = 1 -|x-y|. 
$$

In [ ]:
DF = op.discrepancy_functional(x=x,y=z,set_codpy_kernel = alg.set_sampler_kernel)
dys = DF.eval(y.reshape(y.shape[0],1,y.shape[1]))
dxs = DF.eval(x.reshape(x.shape[0],1,x.shape[1]))
compare_plot_lists([y,x],[dys,dxs])

Indeed, in view of Figure \@ref(fig:drelu) it is clear that the function $y \mapsto d_k(x,y)$ is piecewise differentiable. Hence in some situations, the functional $d_k(x,y)$ might have an infinity of solutions (here on the "flat" segment). The next figure \@ref(fig:relu2) plots $y:=(y^1,y^2) \mapsto d_k(x,y)$  for the two dimensional case.

In [ ]:
dzs = DF.eval(y2.reshape(y2.shape[0],2,int(y2.shape[1] / 2)))
multi_plot([(y2,dzs)],plot_trisurf,projection='3d', elev = 30)

**A example of continuous kernel: Matern**. The Matern kernel is based upon the following kernel, generating usually functional spaces of continuous functions.
$$
k(x,y) = \exp(-|x-y|)
$$

In [ ]:
DF = op.discrepancy_functional(x=x,y=z,set_codpy_kernel = kernel_setters.set_matern_tensor_kernel)
dys = DF.eval(y.reshape(y.shape[0],1,y.shape[1]))
dxs = DF.eval(x.reshape(x.shape[0],1,x.shape[1]))
compare_plot_lists([y,x],[dys,dxs])

Indeed, in the figure \@ref(fig:dmatern) we see that the functional $y \mapsto d_k(x,y)$ is here almost everywhere concave, and a gradient-descent algorithm can't give good results in this case. The next picture \@ref(fig:dmatern2) plots the two dimensional case $y:=(y^1,y^2) \mapsto d_k(x,y)$ 

In [ ]:
dzs = DF.eval(y2.reshape(y2.shape[0],2,int(y2.shape[1] / 2)))
multi_plot([(y2,dzs)],plot_trisurf,projection='3d',elev=30)